# SensRay Moon model/mesh generation with computation of travel times from sensitivity kernels

This notebook demonstrates the generation of a moon model :
- Creating Moon models (M1, M2, M3, weber_core_smooth, weber_core)
- Creating tetrahedral meshes using model discontinuities and parameters
- Populating seismic properties
- Basic visualization and cross-sections
- Sources, receivers, and phases with rays computed
- Computation and visualization of sensitivity kernels and ray paths
- Sparse G matrix calculation
- Delta V function generation, projection, and visualisation
- Computation of travel times using dv with G

In [1]:
import logging
from typing import Any, Iterable, List, Tuple

import numpy as np
from itertools import product
from typing import Callable, Union
import quadpy
from sensray import PlanetModel, CoordinateConverter
from GFwdOpClassLinOp import GFwdOp

## 1. Creation of Moon model and mesh

Creating a model and mesh for ray tracing

### Creating model

Creating a moon model from information stored in .nd file
- User-defined model (model_name), mesh size, and inner core mesh size
- Model creation of PlanetModel class

In [2]:
# User-defined parameters
model_name = "M1"
mesh_size_km = 1000
mesh_size_inner_km = 600

# Model creation
model = PlanetModel(f'../../sensray/models/{model_name}.nd', model_name)

### Creating layered tetrahedral mesh

Uses a layered tetrahedral mesh to control resolution across discontinuities
- User-defined mesh name
- Radii and H_layers gathered from the model properties
- Populate mesh properties with vp, vs, rho from the file (also gathered from model)

In [3]:
# Create mesh and save if not exist, otherwise load existing
mesh_name = "M1_mesh"

try:
    model.create_mesh(from_file=mesh_name)
    print(f"Loaded existing mesh from {mesh_name}")
except FileNotFoundError:
    print("Creating new mesh...")
    radii = model.get_discontinuities()
    H_layers = [mesh_size_km]*(len(radii)-1)+[mesh_size_inner_km]
    model.create_mesh(mesh_size_km=mesh_size_km, radii=radii, H_layers=H_layers)
    model.mesh.populate_properties(model.get_info()["properties"])
    model.mesh.save(f"{model_name}_mesh")  # Save mesh to VT
print(f"Created mesh: {model.mesh.mesh.n_cells} cells")

Loaded mesh from M1_mesh.vtu
Loaded metadata: 9975 cells, 2258 points
Loaded existing mesh from M1_mesh
Created mesh: 9975 cells


## 2. Define Source-Receiver Geometry

Set up realistic earthquake-station-phase and translate to earthquake-station-ray combination(s).

In practise this will be created and filtered individually by the user, then passed into the forward operator. 

### Point generation

Function to generate a point in the model with optional parameters for min/max point type, latitude, longitde, and depth. Alternatively using predefined single source-receiver-phase

In [4]:
# function to make points
def point(pointType="Source", minLat=-90, maxLat=90, minLon=-180, maxLon=180, minDepth=0, maxDepth=700):
    '''
    pointType: "Source" or "Receiver"
    For Source: lat, lon, depth
    For Receiver: lat, lon
    returns: (lat, lon, depth) or (lat, lon)
    '''
    if pointType == "Source":
        lat = np.random.uniform(minLat, maxLat)
        lon = np.random.uniform(minLon, maxLon)
        depth = np.random.uniform(minDepth, maxDepth)  # depth in km
        return (lat, lon, depth)
    elif pointType == "Receiver":
        lat = np.random.uniform(minLat, maxLat)
        lon = np.random.uniform(minLon, maxLon)
        return (lat, lon)
    else:
        raise ValueError("pointType must be 'Source' or 'Receiver'")


print("Source-receiver-phase list generation...")
# Generate source and receiver points and combinations
# sources = [point("Source", minDepth=150, maxDepth=150) for _ in range(2)]
# receivers = [point("Receiver", maxDepth=0) for _ in range(5)]
# phases = ["P", "S", "ScS"]
# srp = [prod + tuple([phases]) for prod in product(sources, receivers)]

# testing with one source-receiver pair - same as initial test
source_lat, source_lon, source_depth = 0.0, 0.0, 150.0  # Equator, 150 km depth
receiver_lat, receiver_lon = 30.0, 45.0  # Surface station
srp = [((source_lat, source_lon, source_depth), (receiver_lat, receiver_lon), ["P", "S", "ScS"])]

Source-receiver-phase list generation...


### Ray tracing with taup

Function to generate a ray using taup for each phase of interest for each source-receiver pair. Save as a list of lists of [source-receiver-ray] to be passed into the forward operatot

In [5]:
def get_rays(srp):
    '''
    srp: list of tuples (source, receiver, phases)
    where source = (lat, lon, depth), receiver = (lat, lon), phases = [phase1, phase2, ...]
    returns array of (source, receiver, ray) for each ray
    '''
    srr_list = []
    for (source, receiver, phases) in srp:
        rays = model.taupy_model.get_ray_paths_geo(
            source_depth_in_km=source[2],
            source_latitude_in_deg=source[0],
            source_longitude_in_deg=source[1],
            receiver_latitude_in_deg=receiver[0],
            receiver_longitude_in_deg=receiver[1],
            phase_list=phases,
        )
        for ray in rays:
            srr_list.append((source, receiver, ray))

    return np.array(srr_list, dtype=object)


print("Compute ray for each source-receiver-phase combination...")
srr = get_rays(srp)

Compute ray for each source-receiver-phase combination...
Building obspy.taup model for '../../sensray/models/M1.nd' ...
filename = ../../sensray/models/M1.nd
Done reading velocity model.
Radius of model . is 1737.1
Using parameters provided in TauP_config.ini (or defaults if not) to call SlownessModel...
Parameters are:
taup.create.min_delta_p = 0.1 sec / radian
taup.create.max_delta_p = 11.0 sec / radian
taup.create.max_depth_interval = 115.0 kilometers
taup.create.max_range_interval = 0.04363323129985824 degrees
taup.create.max_interp_error = 0.05 seconds
taup.create.allow_inner_core_s = True
Slow model  643 P layers,747 S layers
Done calculating Tau branches.
Done Saving /tmp/M1.npz
Method run is done, but not necessarily successful.


## 3. Compute sensitivity kernels and G matrix

Compute travel-time sensitivity kernels: K = -L / v² for each ray using GFwdOp(LinearOperator) and store as a sparse G matrix

In [6]:
print("Calculate sensitivity kernels and sparse G matrix...")
G = GFwdOp(model, srr[:,2])

Calculate sensitivity kernels and sparse G matrix...
Stored sensitivity kernel as cell data: 'K_P_vp'
Stored sensitivity kernel as cell data: 'K_S_vs'
Stored sensitivity kernel as cell data: 'K_ScS_vs'
Stored sensitivity kernel as cell data: 'K_S_vs'
Stored sensitivity kernel as cell data: 'K_ScS_vs'
P Kernel Matrix shape: (1, 9975), nnz: 24
S Kernel Matrix shape: (2, 9975), nnz: 59
ScS Kernel Matrix shape: (2, 9975), nnz: 122


# 4. Create Dv and apply to G

Create vectorized scalar field from radial and angular functions R(r), T(θ,φ) then project onto the mesh. Apply Dv to G to compute travel times.

### Scalar field generation

Function to convert radial and angular functions to a vectorized scalar function in f(x,y,z)

In [7]:
Number = Union[float, int]

def make_scalar_field(
    R: Callable[[np.ndarray], np.ndarray],
    T: Callable[[np.ndarray, np.ndarray], np.ndarray],
) -> Callable[[Union[np.ndarray, list, tuple, Number], Union[Number, None], Union[Number, None]], np.ndarray]:
    """
    Creates a scalar field f(x, y, z) = R(r) * T(theta, phi),
    vectorized to work with quadpy (input shape (3, n_points)).

    Parameters
    ----------
    R : callable
        Radial function R(r)
    T : callable
        Angular function T(theta, phi)

    Returns
    -------
    f : callable
        Vectorized scalar field f(x, y, z)
    """

    def f(x, y=None, z=None):
        # Case 1: quadpy-style call, x is (3, n_points)
        if y is None and z is None:
            x = np.asarray(x)
            # Accept (3, n_points) or (n_points, 3)
            if x.ndim == 2 and x.shape[0] == 3:
                X, Y, Z = x[0], x[1], x[2]
            elif x.ndim == 2 and x.shape[1] == 3:
                X, Y, Z = x[:, 0], x[:, 1], x[:, 2]
            elif x.shape == (3,):
                X, Y, Z = x
            else:
                raise ValueError(f"Unexpected input shape: {x.shape}")
        else:
            # Case 2: standard call f(x, y, z)
            X = np.asarray(x)
            Y = np.asarray(y)
            Z = np.asarray(z)

        # Convert Cartesian to spherical
        r = np.sqrt(X**2 + Y**2 + Z**2)
        theta = np.where(r == 0, 0.0, np.arccos(np.clip(Z / r, -1.0, 1.0)))
        phi = np.mod(np.arctan2(Y, X), 2*np.pi)

        return R(r) * T(theta, phi)

    return f

print(f"Scalar function generation...")
# make function for velocity perturbation
# Define R(r) and T(theta, phi)
# R = lambda r: r**2 * np.exp(-r/100000)
# T = lambda theta, phi: np.cos(theta)

R = lambda r: np.ones_like(r)
T = lambda theta, phi: np.ones_like(theta)

f = make_scalar_field(R, T)

Scalar function generation...


### Dv mesh projection and visualisation

Project scalar field onto the mesh to calculate dv

In [8]:
print("Dv projection onto mesh...")
model.mesh.project_function_on_mesh(f, property_name="dv")

Dv projection onto mesh...


AttributeError: 'PlanetMesh' object has no attribute 'project_function_on_mesh'

Visualise dv field using function and first source-receiver pair for sphere slice

In [ ]:
def display_dv(source_lat, source_lon, receiver_lat, receiver_lon):
    '''
    Display the dv property on a cross-section plane defined by source and receiver
    '''
    plane_normal = CoordinateConverter.compute_gc_plane_normal(
        source_lat, source_lon, receiver_lat, receiver_lon
    )

    # Cross-section showing background Vp
    print("Projected Dv velocity:")
    plane_normal = plane_normal
    plotter1 = model.mesh.plot_cross_section(
        plane_normal=plane_normal,
        property_name='dv',
    )
    plotter1.camera.position = (8000, 6000, 10000)
    plotter1.show()


print("Dv visualization...")
display_dv(srr[0,0][0], srr[0,0][1], srr[0,1][0], srr[0,1][1])

### Time travel computation

Apply dv to G forward operator to compute travel times.

In [ ]:
print("Time travel computation...")
travel_times = G.__apply__(model.mesh.mesh.cell_data["dv"])
print(travel_times)